# 🤖 Breadboard AI Tutor (Student Chatbot)

Welcome!

This chatbot will help you learn breadboard experiments.

### ✅ What you need to do:
1. Run the setup cell below  
2. Enter the Gemini API key (teacher will provide it)  
3. Start asking questions in the chat window  

Type **exit** anytime to stop.

In [ ]:
!pip install -q google-genai gradio requests

In [ ]:
import json, requests
from getpass import getpass
from google import genai

print("✅ Loading Breadboard Experiment Dataset from GitHub...")

DATA_URL = "https://raw.githubusercontent.com/PranavOaR/breadboard-ai-tutor/main/data/experiments.json"

experiments = requests.get(DATA_URL).json()

print("✅ Experiments Loaded:", len(experiments))
print("✅ Experiment Names:", [e["experiment_name"] for e in experiments])

# Secure Gemini API key input
GEMINI_API_KEY = getpass("Enter valid Gemini API Key from AI Studio: ")

# Initialize Gemini client (new SDK)
client = genai.Client(api_key=GEMINI_API_KEY)

# Test the API key
try:
    test = client.models.generate_content(
        model="gemini-2.0-flash",
        contents="Say: Gemini is working!"
    )
    print("✅ Gemini Key Valid!")
    print("✅ Gemini Tutor Ready!")
except Exception as e:
    print("❌ Gemini Key Invalid:", e)
    print("⚠ Chatbot will use dataset-only fallback mode.")

In [ ]:
def safe_gemini_generate(prompt, fallback_text):
    """Safe wrapper for Gemini API calls with silent fallback"""
    try:
        response = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
        return response.text
    except:
        return fallback_text

print("✅ Safe Gemini Wrapper Defined (with silent fallback)")

In [ ]:
SYSTEM_PROMPT = """
You are a Breadboard Tutor Chatbot.

Rules:
- Answer ONLY using the dataset context provided.
- Do NOT use external knowledge.
- Always explain like a teacher for students.
- If answer is not found, say:
  "Sorry, I can only answer from the breadboard lessons."
"""

def retrieve_context(message):
    """
    Retrieve relevant context from the experiments dataset
    """
    message_lower = message.lower()
    context = ""

    # Search experiments by name
    for exp in experiments:
        exp_name_lower = exp["experiment_name"].lower()
        
        # Check if any word from the message matches the experiment name
        if any(word in exp_name_lower for word in message_lower.split()) or exp_name_lower in message_lower:
            context += f"\n✅ Experiment: {exp['experiment_name']}\n"
            context += f"Objective: {exp['objective']}\n\n"
            context += "Steps:\n"
            context += "\n".join(exp["steps"])
            context += "\n"

    # Search components
    for exp in experiments:
        for comp, info in exp["component_working"].items():
            if comp.lower() in message_lower:
                context += f"\n🔧 Component: {comp}\nExplanation: {info}\n"

    return context.strip()


def breadboard_tutor(message, history):
    """
    Main chatbot function for Gradio ChatInterface
    
    Args:
        message: User's question (string)
        history: Chat history (list) - required by Gradio ChatInterface
    
    Returns:
        Response string
    """
    # Special handler: show experiments list
    if "show" in message.lower() and "experiment" in message.lower():
        exp_names = "\n".join(
            [f"- {exp['experiment_name']}" for exp in experiments]
        )
        return f"✅ Available Experiments:\n{exp_names}"

    # Retrieve dataset context
    context = retrieve_context(message)

    # Fallback if nothing matches
    if context == "":
        return "Sorry, I can only answer from the breadboard lessons. Try asking:\n- 'Show me experiments'\n- About specific experiments like '2-Pin LED'\n- About components like 'What is a potentiometer?'"

    # Build grounded prompt
    prompt = f"""
{SYSTEM_PROMPT}

DATASET CONTEXT:
{context}

STUDENT QUESTION:
{message}

Now respond clearly with step-by-step help.
"""

    # ⚠ Use safe wrapper with dataset fallback
    fallback = f"📚 Here's what I found from the experiments:\n\n{context}\n\nYou can explore this information for your question."
    
    return safe_gemini_generate(prompt, fallback)

print("✅ Chatbot Functions Defined")

In [ ]:
import gradio as gr

print("🚀 Launching Student Chatbot Interface...")

demo = gr.ChatInterface(
    fn=breadboard_tutor,
    title="🤖 Breadboard AI Tutor",
    description="Ask about LEDs, buzzers, motors, breadboards, and troubleshooting.",
    examples=[
        "Show me experiments",
        "Explain 2-Pin LED Breadboard Experiment",
        "What is an LDR?",
        "Why is my LED not glowing?"
    ],
    type="messages"
)

demo.launch(share=True, debug=True)